# Guardrails in Strands SDK
Building safe and responsible AI applications is crucial, especially in sensitive domains like finance, healthcare, and customer service. Agentic AI systems can benefit significantly from guardrails that help ensure compliance with safety and ethical standards.

[Strands Agents](https://github.com/strands-agents/sdk-python) is a simple yet powerful SDK that takes a model-driven approach to building and running AI agents. From simple conversational assistants to complex autonomous workflows, from local development to production deployment, Strands Agents scales with your needs.

[Amazon Bedrock Guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html) provide safety mechanisms that help control AI system behavior by defining boundaries for content generation and interaction. The Strands Agents SDK offers seamless integration with these guardrails, enabling you to implement:

* **Content filtering** - Block harmful or inappropriate content
* **Topic blocking** - Prevent discussions on specific topics
* **PII protection** - Detect and handle personally identifiable information
* **Word and phrase filtering** - Control specific language in interactions
* **Contextual grounding** - Ensure responses are relevant and factual

In this notebook, we will explore how to use Amazon Bedrock Guardrails with Strands Agents SDK to ensure safe and responsible AI interactions in a banking assistant application.


## 1.Setup

### 1.1 Install required libraries

In [ ]:
import os
import sys
import json
import logging
from typing import Dict, List, Any, Tuple, Optional
from strands import Agent, tool
from strands.models import BedrockModel
import requests
from bs4 import BeautifulSoup
import boto3
import random

# Set up environment variables for AWS before importing Strands
os.environ["AWS_PROFILE"] = "default"
os.environ["AWS_REGION"] = "us-east-1"

logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s | %(name)s | %(message)s"
)

### 1.2 Set up the client

In [ ]:
client = boto3.client("bedrock")

## 2. Create a Guardrail
We will now create our first Guardrail, which will prevent the model from providing fiduciary advice. Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. You can also review the [API documentation](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_CreateGuardrail.html#bedrock-CreateGuardrail-request-contextualGroundingPolicyConfig) for more details.

In [ ]:
#If the guardrail already exists, it will return the existing guardrail.
list_response = client.list_guardrails(maxResults=10)

exists = False
for guardrail in list_response.get('guardrails', []):
    if guardrail['name'] == 'financial-advice':
        print(f"Guardrail already exists: {guardrail['id']}")
        guardrail_id = guardrail['id']
        exists = True
if not exists:

    create_response = client.create_guardrail(
        name='financial-advice',
        description='Prevents the our model from providing financial advice.',
        topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Fiduciary Advice',
                    'definition': 'Provide advice on financial investments.',
                    'examples': [
                        'What stocks should I invest in for my retirement?',
                        'Is it a good idea to put my money in a mutual fund?',
                        'How should I allocate my 401(k) investments?',
                        'What type of trust fund should I set up for my children?',
                        'Should I hire a financial advisor to manage my investments?'
                    ],
                    'type': 'DENY'
                }
            ]
        },
        contentPolicyConfig={
            'filtersConfig': [
                {
                    'type': 'SEXUAL',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'VIOLENCE',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'HATE',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'INSULTS',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'MISCONDUCT',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'HIGH'
                },
                {
                    'type': 'PROMPT_ATTACK',
                    'inputStrength': 'HIGH',
                    'outputStrength': 'NONE'
                }
            ]
        },
        wordPolicyConfig={
            'wordsConfig': [
                {'text': 'fiduciary advice'},
                {'text': 'investment recommendations'},
                {'text': 'stock picks'},
                {'text': 'financial planning guidance'},
                {'text': 'portfolio allocation advice'},
                {'text': 'retirement fund suggestions'},
                {'text': 'wealth management tips'},
                {'text': 'trust fund setup'},
                {'text': 'investment strategy'},
                {'text': 'financial advisor recommendations'}
            ],
            'managedWordListsConfig': [
                {'type': 'PROFANITY'}
            ]
        },
        sensitiveInformationPolicyConfig={
            'piiEntitiesConfig': [
                {'type': 'EMAIL', 'action': 'ANONYMIZE'},
                {'type': 'PHONE', 'action': 'ANONYMIZE'},
                {'type': 'NAME', 'action': 'ANONYMIZE'},
                {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
                {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
                {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
            ],
            'regexesConfig': [
                {
                    'name': 'Account Number',
                    'description': 'Matches account numbers in the format XXX123456',
                    'pattern': r'\b\d{6}\d{4}\b',
                    'action': 'ANONYMIZE'
                }
            ]
        },
        contextualGroundingPolicyConfig={
            'filtersConfig': [
                {
                    'type': 'GROUNDING',
                    'threshold': 0.75
                },
                {
                    'type': 'RELEVANCE',
                    'threshold': 0.75
                }
            ]
        },
        blockedInputMessaging="""Sorry, contact our customer service for this request.""",
        blockedOutputsMessaging="""Sorry, contact our customer service for this request. """,
        tags=[
            {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
            {'key': 'environment', 'value': 'production'}
        ]
    )

    print(create_response)
    guardrail_id = create_response['guardrailId']

## 3. Create the tools for our agent
We will now create the tools that our agent will be able to access during the conversation. In this case, we are creating a Banking assistant agent, therefore we will create the following tools:
* `get_balance`: A tool to get the balance of a bank account.
* `find_branch`: A tool to find the nearest bank branch.
* `check_loan_status`: A tool to check the status of a loan application.

We can use the tool decorator to create these tools. Strands SDK also supports Model Context Protocol (MCP).


We will first create a sqlite database to store the banking information, and then create the tools that will interact with this database.



In [ ]:
from db_build import setup_bank_database
import sqlite3

In [ ]:
setup_bank_database()
DB_PATH = 'data/bank_data.db'

In [ ]:

@tool
def get_loan_status(account_id):
   """
   Get all loan information for a specific account ID
   
   Args:
       account_id (str): The account ID to lookup
       
   Returns:
       list: List of dictionaries containing loan information, empty list if none found
             Format: [{'id': int, 'account_id': str, 'loan_amount': float, 
                      'interest_rate': float, 'status': str, 'last_updated': str}]
   """
   conn = sqlite3.connect(DB_PATH)
   cursor = conn.cursor()
   
   try:
       cursor.execute('''
           SELECT id, account_id, loan_amount, interest_rate, status, last_updated 
           FROM loan_status 
           WHERE account_id = ?
           ORDER BY last_updated DESC
           LIMIT 1
       ''', (account_id,))
       
       results = cursor.fetchall()
       
       loans = []
       for row in results:
           loans.append({
               'id': row[0],
               'account_id': row[1],
               'loan_amount': row[2],
               'interest_rate': row[3],
               'status': row[4],
               'last_updated': row[5]
           })
       
       return loans
       
   except sqlite3.Error as e:
       print(f"Database error: {e}")
       return []
   finally:
       conn.close()


In [ ]:
@tool
def get_account_balance(account_id):
    """
    Get the account balance for a specific account ID
    
    Args:
        account_id (str): The account ID to lookup
        
    Returns:
        dict: Dictionary containing account information or None if not found
              Format: {'account_id': str, 'balance': float, 'last_updated': str}
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    try:
        cursor.execute('''
            SELECT account_id, balance, last_updated 
            FROM account_balance 
            WHERE account_id = ?
            ORDER BY last_updated DESC
            LIMIT 1
        ''', (account_id,))
        
        result = cursor.fetchone()
        
        if result:
            return {
                'account_id': result[0],
                'balance': result[1],
                'last_updated': result[2]
            }
        else:
            return None
            
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        conn.close()

In [ ]:
@tool
def get_nearest_branch(zip_code):
   """
   Get the nearest bank branch based on zip code
   
   Args:
       zip_code (str): The zip code to find nearest branch for
       
   Returns:
       dict: Dictionary containing branch information
             Format: {'branch_id': str, 'name': str, 'address': str, 'city': str, 
                     'state': str, 'zip_code': str, 'phone': str, 'distance_miles': float,
                     'hours': str, 'services': list}
   """
   # Mock branch data - in reality this would come from a database or API
   mock_branches = {
       # Major city zip codes and their branches
       '10001': {'branch_id': 'NYC001', 'name': 'Manhattan Central Branch', 'address': '123 Broadway', 'city': 'New York', 'state': 'NY'},
       '90210': {'branch_id': 'LA001', 'name': 'Beverly Hills Branch', 'address': '456 Rodeo Drive', 'city': 'Beverly Hills', 'state': 'CA'},
       '60601': {'branch_id': 'CHI001', 'name': 'Downtown Chicago Branch', 'address': '789 Michigan Ave', 'city': 'Chicago', 'state': 'IL'},
       '77001': {'branch_id': 'HOU001', 'name': 'Houston Main Branch', 'address': '321 Main Street', 'city': 'Houston', 'state': 'TX'},
       '33101': {'branch_id': 'MIA001', 'name': 'Miami Beach Branch', 'address': '654 Ocean Drive', 'city': 'Miami', 'state': 'FL'},
       '98101': {'branch_id': 'SEA001', 'name': 'Seattle Downtown Branch', 'address': '987 Pine Street', 'city': 'Seattle', 'state': 'WA'},
       '02101': {'branch_id': 'BOS001', 'name': 'Boston Financial District', 'address': '147 State Street', 'city': 'Boston', 'state': 'MA'},
       '30301': {'branch_id': 'ATL001', 'name': 'Atlanta Midtown Branch', 'address': '258 Peachtree St', 'city': 'Atlanta', 'state': 'GA'},
   }
   
   # Check if we have an exact match
   if zip_code in mock_branches:
       branch_data = mock_branches[zip_code]
       distance = round(random.uniform(0.5, 2.0), 1)  # Very close for exact zip match
   else:
       # For unknown zip codes, return a random nearby branch
       branch_data = random.choice(list(mock_branches.values()))
       distance = round(random.uniform(2.5, 15.0), 1)  # Further distance for non-exact matches
   
   # Generate additional branch details
   services = random.sample([
       'ATM', 'Drive-through', 'Safe Deposit Boxes', 'Notary Services',
       'Business Banking', 'Mortgage Services', 'Investment Consulting',
       'Currency Exchange', '24/7 Banking', 'Mobile Banking Support'
   ], k=random.randint(4, 7))
   
   hours_options = [
       'Mon-Fri: 9AM-5PM, Sat: 9AM-2PM',
       'Mon-Fri: 8AM-6PM, Sat: 9AM-3PM',
       'Mon-Thu: 9AM-4PM, Fri: 9AM-6PM, Sat: 9AM-1PM',
       '24/7 ATM Access, Lobby: Mon-Fri 9AM-5PM'
   ]
   
   return {
       'branch_id': branch_data['branch_id'],
       'name': branch_data['name'],
       'address': branch_data['address'],
       'city': branch_data['city'],
       'state': branch_data['state'],
       'zip_code': zip_code,
       'phone': f"({random.randint(200, 999)}) {random.randint(200, 999)}-{random.randint(1000, 9999)}",
       'distance_miles': distance,
       'hours': random.choice(hours_options),
       'services': services
   }

## 4. Creating the Agent

We will now create a Strands agent that will use the guardrail we created earlier. The agent will be a Banking assistant that can answer questions about banking services, check account balances, find bank branches, and check loan statuses. 

There are two ways to integrate guardrails with Strands agents:
1. If you are using Amazon Bedrock Model, a built-in framework integrates directly with the guardrails. If a guardrail is triggered, the Strands Agents SDK will automatically overwrite the user's input in the conversation history. This is done so that follow-up questions are not also blocked by the same questions. This can be configured with the guardrail_redact_input boolean, and the guardrail_redact_input_message string to change the overwrite message. Additionally, the same functionality is built for the model's output, but this is disabled by default. You can enable this with the guardrail_redact_output boolean, and change the overwrite message with the guardrail_redact_output_message.
2. If you are using a different model, you can use the Apply Guardrail API, which is agnostic to the model you are using. This API will apply the guardrail to the input and output of the model, and will return a modified input and output if the guardrail is triggered.

### 4.1 Create the agent and use built-in guardrail integration
We will first use Bedrock Model with the built-in guardrail integration


In [ ]:

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    guardrail_id=guardrail_id,         # Your Bedrock guardrail ID
    guardrail_version="DRAFT",                    # Guardrail version
    guardrail_trace="enabled",                # Enable trace info for debugging
)

# Create agent with the guardrail-protected model
agent = Agent(
    system_prompt="You are a helpful banking assistant. You can answer questions about account balances, loan statuses, and branch locations. You must not provide fiduciary advice or personalized financial recommendations.",
    tools=[get_loan_status, get_account_balance, get_nearest_branch, retrieve_faq],
    model=bedrock_model,
)



In [ ]:
while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting the chat. Goodbye!")
            break
        
        
        # Use the agent with the guardrailed input
        response = agent(user_input)
        
        # Print the agent's response
        print(f"Agent: {response}")
    
    except KeyboardInterrupt:
        print("\nExiting the chat. Goodbye!")
        break
    except Exception as e:
        logging.error(f"An error occurred: {e}")

## 4.2 Create the agent and use Apply Guardrail API
We will now use the Apply Guardrail API.


In [ ]:

def apply_guardrails(content):
    # Type: (str) -> Tuple[str, Dict[str, Any]]
    """
    Apply AWS Bedrock Guardrails to content
    
    Args:
        content: Text content to analyze with guardrails
        
    Returns:
        Tuple containing guardrailed content and metadata
    """
    try:
        logging.info(f"Applying guardrail to content: {content[:50]}...")
        
        response = client.apply_guardrail(
            guardrailIdentifier=guardrail_id,
            guardrailVersion="DRAFT",
            source="INPUT",
            content=[
                {
                    "text": {
                        "text": content
                    }
                }
            ]
        )
        
        # Log the full response for debugging
        import json
        logging.info(f"Full guardrail response: {json.dumps(response, default=str, indent=2)}")
        
        # Parse response
        outputs = response.get('outputs', [])
        if outputs and len(outputs) > 0:
            guardrailed_content = outputs[0].get('text', content)
        else:
            guardrailed_content = content
        
        # Get guardrail action - this is the official way to determine if guardrails intervened
        # GUARDRAIL_INTERVENED - Guardrail modified or blocked the content
        # NONE - Guardrail did not modify or block the content
        action = response.get('action', 'NONE')
        
        # Check if the content was actually changed
        content_changed = guardrailed_content != content
        
        # Log more detailed information
        if action == "GUARDRAIL_INTERVENED":
            logging.info(f"Guardrail intervention details:")
            logging.info(f"- Action: {action}")
            logging.info(f"- Content actually changed: {content_changed}")
            logging.info(f"- Original content length: {len(content)}")
            logging.info(f"- Guardrailed content length: {len(guardrailed_content)}")
            
            # Log specific assessment details
            if 'assessments' in response:
                for idx, assessment in enumerate(response['assessments']):
                    logging.info(f"- Assessment #{idx+1}:")
                    if 'topicPolicy' in assessment:
                        topic_policy = assessment['topicPolicy']
                        if 'topics' in topic_policy:
                            for topic in topic_policy['topics']:
                                logging.info(f"  - Topic: {topic.get('name', 'Unknown')}")
                                logging.info(f"  - Type: {topic.get('type', 'Unknown')}")
                                logging.info(f"  - Action: {topic.get('action', 'Unknown')}")
                    
                    if 'contentPolicy' in assessment:
                        content_policy = assessment['contentPolicy']
                        if 'filters' in content_policy:
                            for filter_item in content_policy['filters']:
                                logging.info(f"  - Filter Type: {filter_item.get('type', 'Unknown')}")
                                logging.info(f"  - Filter Action: {filter_item.get('action', 'Unknown')}")
        
        # Create metadata dictionary with the complete response
        metadata_dict = {
            "action": action,
            "assessments": response.get('assessments', []),
            "content_changed": content_changed
        }
        
        return guardrailed_content, metadata_dict
    
    except Exception as e:
        logging.error(f"Error applying guardrails: {e}")
        import traceback
        traceback.print_exc()
        return content, {"action": "ERROR", "error": str(e), "assessments": []}



In [ ]:
# Create the agent 
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
)
agent_v2 = Agent(
    system_prompt="You are a helpful banking assistant. You can answer questions about account balances, loan statuses, and branch locations. You must not provide fiduciary advice or personalized financial recommendations.",
    tools=[get_loan_status, get_account_balance, get_nearest_branch],
    model=bedrock_model,
)



We will apply guardrails to the input (i.e the user query), however the same can be done for the output (i.e the model response) as well. The Apply Guardrail API will return a modified input and output if the guardrail is triggered.

In [ ]:
while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting the chat. Goodbye!")
            break
        
        # Apply guardrails to the user input
        guardrailed_input, metadata = apply_guardrails(user_input)
        
        # Log the guardrail metadata
        logging.info(f"Guardrail metadata: {json.dumps(metadata, indent=2)}")
        if metadata['action'] == "GUARDRAIL_INTERVENED":
            logging.warning("Guardrail intervention detected! ")
            break
        
        # Use the agent with the guardrailed input
        response = agent_v2(guardrailed_input)
        
        # Print the agent's response
        print(f"Agent: {response}")
    
    except KeyboardInterrupt:
        print("\nExiting the chat. Goodbye!")
        break
    except Exception as e:
        logging.error(f"An error occurred: {e}")